# 120 Years of Olympic History: Data Wrangling and Visualisation in Pandas

**Module:** MN5813 – Business Analytics  
**Assessment:** Group Report (20%) – Olympics Data Wrangling & Visualisation  
**Candidate Number:** 2610587  

**GitHub repository (notebook + files):**  
https://github.com/Grewal226/olympics_project_report_cleean  

---


## Access to Notebook and Data (QR code & URL)

Markers can access the original Jupyter Notebook and associated files (including the data file and any exported figures) via the following URL:

> https://github.com/Grewal226/olympics_project_report_cleean

The QR code below also links directly to the same repository.


In [ ]:
# QR code generation (optional but recommended for PDF version)

repo_url = "https://github.com/Grewal226/olympics_project_report_cleean"

try:
    import qrcode
    from IPython.display import Image, display

    qr_img = qrcode.make(repo_url)
    qr_path = "qr_github_repo.png"
    qr_img.save(qr_path)
    display(Image(filename=qr_path))
    print(f"QR code saved as {qr_path}")
except ImportError:
    print("qrcode library is not installed. To generate the QR code, run:")
    print("!pip install qrcode[pil]")
    print("Then re-run this cell.")


## AI Use Statement

This notebook was prepared with the assistance of a generative AI tool (OpenAI ChatGPT, GPT‑5.1 Thinking) for help with code structure,
markdown wording and general organisation. All code cells have been reviewed, tested and, where necessary, adapted by the author, who takes
responsibility for the final content, interpretation and any errors that remain.


## 1. Aim and objectives

**Aim**  
To demonstrate how the Pandas library can be used to clean, transform and visually explore a large real-world dataset, using 120 years of
Olympic athlete–event records as a case study.

**Objectives**  

1. To load the Olympic athlete–event dataset into a Jupyter Notebook environment and manage it using Pandas DataFrames.
2. To clean and prepare the data for analysis, including handling missing values, removing duplicates and constructing additional variables
   that make the data easier to work with.
3. To apply a set of exploratory techniques – including grouping operations and visualisation – to highlight patterns in athlete
   demographics, medal distributions and participation over time, and to summarise the main findings.


## 2. Context and tools

### 2.1 Brief context and literature background

Olympic results are often used in teaching examples and in applied research because they sit at the intersection of sport, economics and
politics. A common theme in the literature is that medal counts are heavily concentrated in a relatively small number of (usually richer) countries,
although the exact pattern changes over time as new nations emerge and the Games themselves evolve. Other work looks at age profiles
and participation by sex, showing that some sports are dominated by young athletes while others allow longer careers, and that the share of
female competitors has risen steadily over the twentieth century.

This notebook does not try to replicate those studies in full. Instead, the focus is on using the same underlying data to demonstrate sound
data-management practice: making the dataset easier to handle, producing sensible summary statistics and then turning those into visual
summaries that would be understandable to a non-technical audience.

### 2.2 Tools and techniques

The main tools and techniques used in this notebook are:

- **Pandas**: for storing the data in DataFrames, handling missing values, grouping and aggregating records, and creating new variables.
- **Matplotlib** and **Seaborn**: for producing static plots such as bar charts, boxplots and line charts that are suitable for inclusion in a
  written report or slide deck.
- **Jupyter Notebook**: for combining code, output and written commentary in a single, reproducible document.
- **qrcode** (optional): for generating a QR code that links directly to the GitHub repository holding the notebook and associated files.

Together, these tools form a standard workflow for basic data wrangling and visual exploration in Python.


## 3. Data collection

The analysis is based on the widely used *120 years of Olympic history: athletes and results* dataset, originally made available via the
Kaggle platform. The dataset contains athlete–event level records for the modern Olympic Games from Athens 1896 to Rio 2016. Each row
represents one athlete's participation in a particular event in a particular Games.

The dataset is appropriate for this assignment for three main reasons:

- it is large enough (over 270,000 rows) to be realistic, but still manageable in a teaching context;
- it combines demographic information (age, height, weight, sex) with event and medal outcomes, which makes it possible to explore several
  different kinds of patterns; and
- it has been widely used in teaching materials, which means that its structure and limitations are relatively well understood.

For the purposes of this notebook, a local copy of the dataset is stored in the file `athlete_events.csv` in the same directory as the
notebook. The same file is also included in the submission package so that markers can download and execute the notebook without needing
to source the data separately.


In [ ]:
# 4. Data loading

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set some basic plotting defaults
plt.rcParams["figure.figsize"] = (8, 5)
sns.set_theme()

# Load the Olympic athlete–event data
olympics = pd.read_csv("athlete_events.csv")

print("Shape (rows, columns):", olympics.shape)
print("Column names:", olympics.columns.tolist())

olympics.head()


The file `athlete_events.csv` is loaded into a DataFrame called `olympics`. Each row represents a single athlete–event appearance, so the
same athlete can appear in multiple rows if they competed in several events or in more than one Games. The dataset contains 271,116 rows
and 15 columns covering identifiers, demographics, country codes (NOCs), events and medal outcomes.


In [ ]:
# 5. Data processing: cleaning

# 5.1 Overview of missing values
missing_counts = olympics.isna().sum().sort_values(ascending=False)
missing_counts


In [ ]:
# 5.2 Remove exact duplicate rows (if any)
before_rows = len(olympics)
olympics = olympics.drop_duplicates()
after_rows = len(olympics)
removed_duplicates = before_rows - after_rows

print(f"Rows before removing duplicates: {before_rows}")
print(f"Rows after removing duplicates:  {after_rows}")
print(f"Exact duplicate rows removed:    {removed_duplicates}")


In [ ]:
# 5.3 Convert Year to a datetime representation for convenience
olympics["Year_dt"] = pd.to_datetime(olympics["Year"], format="%Y")

olympics.info()


### 5. Data processing: cleaning

The cleaning stage concentrates on simple but important steps that make later work more reliable:

- The `isna()` summary shows that missing values are concentrated in the `Medal`, `Height`, `Weight` and `Age` columns. Missing medals are
  expected because most athletes do not win a medal. Missing age and anthropometric data are more problematic and are handled in a way that
  depends on the analysis being performed.
- Exact duplicate rows across all columns are removed using `drop_duplicates()`. This avoids counting the same athlete–event twice. In the
  original file there are 1,385 such rows, which are dropped.
- A new column `Year_dt` is created by converting the integer `Year` to a datetime object. This does not change the underlying values but
  makes it easier to work with time-based groupings if needed.

At this stage the dataset is structurally tidy and ready for further processing.


In [ ]:
# 5.4 Simple imputation for height and weight (stretch element)

# Work on copies so that the original columns are preserved
olympics["Height_imputed"] = olympics["Height"]
olympics["Weight_imputed"] = olympics["Weight"]

# Compute median height and weight within Sport-Sex groups
group_height_median = olympics.groupby(["Sport", "Sex"])["Height"].transform("median")
group_weight_median = olympics.groupby(["Sport", "Sex"])["Weight"].transform("median")

# Fill missing values using these medians
olympics["Height_imputed"] = olympics["Height_imputed"].fillna(group_height_median)
olympics["Weight_imputed"] = olympics["Weight_imputed"].fillna(group_weight_median)

olympics[["Sport", "Sex", "Height", "Height_imputed", "Weight", "Weight_imputed"]].head()


For some later summaries it is convenient to have complete height and weight information. Rather than dropping all rows with missing
values, a simple form of imputation is used as a stretch element: for each sport–sex combination the median height and weight are
calculated and used to fill in any gaps. The original `Height` and `Weight` columns are retained so that it remains clear which values are
observed and which are imputed.


In [ ]:
# 6. Data processing: wrangling

# 6.1 Age groups
age_bins = [0, 18, 25, 35, 200]
age_labels = ["0–18", "19–25", "26–35", "36+"]
olympics["Age_Group"] = pd.cut(olympics["Age"], bins=age_bins, labels=age_labels, include_lowest=True)
olympics["Age_Group"] = olympics["Age_Group"].cat.add_categories(["Unknown"])
olympics.loc[olympics["Age"].isna(), "Age_Group"] = "Unknown"

# 6.2 Tidy name fields
olympics["Full_Name"] = olympics["Name"].str.strip()
olympics["Given_Name"] = olympics["Full_Name"].str.split().str[0]
olympics["Family_Name"] = olympics["Full_Name"].str.split().str[-1]

# 6.3 Century and decade
olympics["Century_Number"] = (olympics["Year"] - 1) // 100 + 1

def to_ordinal(n: int) -> str:
    # Simple ordinal helper; suitable for small integers such as centuries
    return "%d%s" % (n, "tsnrhtdd"[(n // 10 % 10 != 1) * (n % 10 < 4) * n % 10 :: 4])

olympics["Century_Label"] = olympics["Century_Number"].apply(to_ordinal)
olympics["Decade"] = (olympics["Year"] // 10) * 10

# 6.4 Medal indicators
olympics["Is_Medalist"] = olympics["Medal"].notna().astype(int)
medal_points = {"Gold": 3, "Silver": 2, "Bronze": 1}
olympics["Medal_Score"] = olympics["Medal"].map(medal_points).fillna(0).astype(int)

olympics[["Age", "Age_Group", "Full_Name", "Century_Label", "Decade", "Is_Medalist", "Medal_Score"]].head()


### 6. Data processing: wrangling

Several additional variables are created to make the data easier to analyse:

- **Age_Group**: athletes are placed into broad age bands (0–18, 19–25, 26–35, 36+). Observations without an age are labelled as
  `Unknown`. This allows age patterns to be compared across sports and over time without relying on exact ages.
- **Full_Name**, **Given_Name** and **Family_Name**: the original `Name` field is cleaned so that names can be used consistently when
  aggregating medals by athlete.
- **Century_Label** and **Decade**: these variables summarise the timing of each Games in a more interpretable way than the raw year and
  make it straightforward to look at long-run trends.
- **Is_Medalist** and **Medal_Score**: these indicators distinguish medallists from non-medallists and assign a simple score to medal
  types (Gold = 3, Silver = 2, Bronze = 1, no medal = 0). This is useful when summarising overall success across different dimensions.

These steps satisfy the core wrangling requirements specified in the brief and also introduce a couple of extra variables that are helpful
for later analysis.


In [ ]:
# 7. Visual data exploration and analysis

# 7.1 Average age of athletes for each Olympic event
age_by_event = (
    olympics
    .dropna(subset=["Age"])
    .groupby("Event", as_index=False)["Age"]
    .mean()
    .rename(columns={"Age": "Average_Age"})
)

# Identify youngest and oldest events
youngest_events = age_by_event.sort_values("Average_Age").head(10)
oldest_events = age_by_event.sort_values("Average_Age").tail(10)

print("Youngest events by average age:")
display(youngest_events)

print("Oldest events by average age:")
display(oldest_events)


In [ ]:
# 7.2 Top 10 countries with the most gold medals

gold_only = olympics[olympics["Medal"] == "Gold"]

gold_by_noc = (
    gold_only
    .groupby("NOC", as_index=False)
    .size()
    .rename(columns={"size": "Gold_Count"})
    .sort_values("Gold_Count", ascending=False)
)

top10_gold_noc = gold_by_noc.head(10)
top10_gold_noc


In [ ]:
# 7.3 Athlete with the most medals in each sport

medal_rows = olympics[olympics["Medal"].notna()]

medals_per_athlete_sport = (
    medal_rows
    .groupby(["Sport", "Full_Name"], as_index=False)
    .size()
    .rename(columns={"size": "Medal_Count"})
)

idx_max = medals_per_athlete_sport.groupby("Sport")["Medal_Count"].idxmax()
top_athletes_by_sport = medals_per_athlete_sport.loc[idx_max].sort_values("Medal_Count", ascending=False)

top_athletes_by_sport.head(15)


## 7. Visual data exploration and analysis

The core exploratory analysis follows the assignment brief and combines grouping operations with basic visualisation.

First, average age is calculated for each event, using only rows where `Age` is available. The youngest events tend to be in women's
swimming and rhythmic gymnastics, where the mean age is in the late teens. At the other end of the scale, some early archery team
competitions and the now-discontinued art competitions have mean ages close to fifty. This confirms the intuitive idea that different
sports place peak physical demands at very different stages of an athlete's life.

Second, gold-medal counts are aggregated by NOC and ranked. The United States sits comfortably at the top of the table, followed by the
Soviet Union and Germany. Great Britain, Italy, France, Sweden, Canada, Hungary and East Germany also appear in the top ten. The presence
of NOCs that no longer exist reminds us that simple medal tables blend together sporting performance and geopolitical history.

Third, within each sport the total number of medals is counted for each athlete. Taking the top performer in each sport highlights familiar
names such as Michael Phelps in swimming, Larysa Latynina in gymnastics and others who have dominated their disciplines over several
Games. This reinforces the idea that Olympic success is highly concentrated, not only at the country level but also at the individual level.


In [ ]:
# 8. Visual data exploration: plots

# 8.1 Top 10 NOCs by gold medals (bar chart)
plt.figure()
sns.barplot(data=top10_gold_noc, x="NOC", y="Gold_Count")
plt.title("Top 10 NOCs by number of gold medals")
plt.xlabel("NOC")
plt.ylabel("Number of gold medals")
plt.tight_layout()
plt.savefig("figure_top10_gold_noc.png", dpi=300)
plt.show()


In [ ]:
# 8.2 Age distribution in selected sports (boxplot)

selected_sports = ["Gymnastics", "Swimming", "Equestrianism", "Shooting"]
age_sport_sample = olympics.dropna(subset=["Age"])
age_sport_sample = age_sport_sample[age_sport_sample["Sport"].isin(selected_sports)]

plt.figure()
sns.boxplot(data=age_sport_sample, x="Sport", y="Age")
plt.title("Distribution of athlete age in selected sports")
plt.xlabel("Sport")
plt.ylabel("Age")
plt.tight_layout()
plt.savefig("figure_age_distribution_selected_sports.png", dpi=300)
plt.show()


In [ ]:
# 8.3 Gender participation over time (line chart)

entries_by_sex = olympics.pivot_table(
    index="Year",
    columns="Sex",
    values="ID",
    aggfunc="count"
)

plt.figure()
for sex_code, label in [("M", "Male"), ("F", "Female")]:
    if sex_code in entries_by_sex.columns:
        plt.plot(entries_by_sex.index, entries_by_sex[sex_code], label=label)

plt.legend()
plt.title("Olympic participation by sex over time")
plt.xlabel("Year")
plt.ylabel("Number of athlete–event entries")
plt.tight_layout()
plt.savefig("figure_gender_participation_over_time.png", dpi=300)
plt.show()


### Interpretation of visual exploration

The three figures are intended to be simple but informative:

- **Top 10 NOCs by gold medals**: the bar chart underlines how skewed the distribution of gold medals is. The United States stands well
  clear of other countries, and there is a steep drop-off after the first few NOCs. This is consistent with the idea that long-run Olympic
  success is concentrated among a relatively small group of nations with the resources and sporting infrastructure to support repeated
  high-level performance.
- **Age distributions in selected sports**: the boxplots for gymnastics and swimming show younger age distributions, with low medians and
  relatively narrow interquartile ranges. Equestrianism and shooting, by contrast, have higher median ages and much wider spreads,
  implying that athletes can remain competitive for longer. The visual makes clear at a glance how the physical and technical demands of
  different sports translate into very different career patterns.
- **Gender participation over time**: the line chart confirms that early Games were overwhelmingly male. Female participation rises slowly at
  first and then more rapidly from the latter half of the twentieth century onwards. Although male entries still slightly exceed female
  entries in the most recent Games in the dataset, the gap has narrowed considerably. This pattern reflects the gradual removal of formal
  barriers to women's participation and changing social attitudes towards women's sport.

Together, these visualisations demonstrate three different exploratory techniques (aggregation by group, distribution comparison and
time-series analysis) and show how they can be combined to tell a coherent story.


In [ ]:
# 9. Export of results

# Save the cleaned and wrangled DataFrame
olympics.to_csv("athlete_events_cleaned.csv", index=False)

print("Cleaned dataset exported as 'athlete_events_cleaned.csv'.")
print("Figures saved as PNG files in the current working directory:")
print(" - figure_top10_gold_noc.png")
print(" - figure_age_distribution_selected_sports.png")
print(" - figure_gender_participation_over_time.png")


## 10. Results and conclusion

The results of the cleaning and exploration stages can be summarised in three main points.

First, athlete age varies substantially by sport and event. Some disciplines – notably women's swimming and rhythmic gymnastics – have very
low mean ages, while others such as equestrian events and historical art and archery competitions have much older competitors. This
confirms the intuition that the physical and technical demands of different sports imply very different typical career lengths.

Second, historical gold medals are heavily concentrated among a small subset of NOCs, with the United States at the top, followed by the
Soviet Union, Germany and a cluster of other European countries. Several of the leading NOCs no longer exist in their original form, which
illustrates how simple medal tables blend together sporting achievement and geopolitical change.

Third, participation by sex has shifted from being almost exclusively male in the early Games to a much more balanced picture in recent
decades. The trend lines show that female participation has grown steadily, particularly from the second half of the twentieth century
onwards, reflecting broader social and institutional changes.

From a data-wrangling perspective, the exercise demonstrates that relatively straightforward cleaning (handling duplicates and missing
values), feature construction (creating age groups and time indicators) and grouping operations can already yield useful insight. The
visualisations help turn those numerical summaries into a narrative that would be accessible to a non-technical reader and could form the
basis for further, more detailed analysis (for example, normalising medal counts by population or focusing on specific sports).


## 11. Limitations and possible extensions

A few limitations of the exercise are worth noting. First, the dataset records medal outcomes but not the underlying performance margin,
so it is not possible to distinguish narrow defeats from decisive wins. Secondly, the Olympic programme has expanded and changed over
time, which affects medal opportunities and makes direct comparisons across eras more complicated than the simple summaries suggest.
Thirdly, the way NOCs map onto countries is not straightforward in all cases; states have split and merged, and some NOCs no longer exist.

Finally, although simple imputation was used for height and weight, more sophisticated approaches could be explored, and the impact of
missing data on particular analyses could be investigated in more detail.

There are several natural extensions. Medal totals could be normalised by population or income to give a sense of performance relative to
resources. The analysis could also be narrowed to focus on a single sport or region, or external data (for example, host-nation effects or
macro-economic indicators) could be added to model the determinants of Olympic success more explicitly.


## 12. References

- Kaggle, *120 years of Olympic history: athletes and results* (dataset used in MN5813 materials).  
- Bowden, J. (2011). *Writing a Report: How to Prepare, Write and Present Really Effective Reports*. How To Books.  

Additional academic references on Olympic performance, gender participation and sport analytics could be incorporated to expand the
literature background if required.
